In [169]:
import numpy as np
import pandas as pd 

import plotly.express as px
import plotly.io as pio
import plotly.graph_objects as go

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import (
    mean_squared_error,
    mean_absolute_error,
    r2_score,
    mean_absolute_percentage_error,
)

from keras.models import Sequential # type: ignore
from keras.layers import LSTM, Dense, Dropout, SimpleRNN # type: ignore
from keras.callbacks import EarlyStopping, ReduceLROnPlateau # type: ignore

pio.renderers.default = "vscode"

In [170]:
data = pd.read_csv("cleaned_data.csv")

data['Date'] = pd.to_datetime(data['Date'])
data.set_index('Date', inplace=True)
data = data.sort_values(by=['City_Encoded', 'Year'])

In [171]:
data.head()

,Year,Max Temperature,Avg Temperature,Min Temperature,Dew Point,Precipitation,Snowdepth,Wind,Gust Wind,Sea Level Pressure,City_Encoded,Month_sin,Month_cos
Date,,,,,,,,,,,,,
2013-01-01,2013,76.81,66.86,56.81,52.39,0.0,0.0,8.21,0.06,29.95,0,0.500000,8.660254e-01
2013-02-01,2013,78.93,68.66,59.57,55.58,0.0,0.0,8.25,0.08,29.90,0,0.866025,5.000000e-01
2013-03-01,2013,86.13,74.63,64.68,53.01,0.0,0.0,8.90,0.12,29.82,0,1.000000,6.123234e-17
2013-04-01,2013,92.63,82.20,68.43,55.99,0.0,0.0,8.76,0.09,29.68,0,0.866025,-5.000000e-01
2013-05-01,2013,98.52,87.53,77.16,62.66,0.0,0.0,9.17,0.13,29.60,0,0.500000,-8.660254e-01


In [172]:
print("Data Shape -->", data.shape)

Data Shape --> (10282, 13)


In [173]:
data.describe()

,Year,Max Temperature,Avg Temperature,Min Temperature,Dew Point,Precipitation,Snowdepth,Wind,Gust Wind,Sea Level Pressure,City_Encoded,Month_sin,Month_cos
count,10282.000000,10282.000000,10282.000000,10282.000000,10282.000000,10282.000000,10282.000000,10282.00000,10282.000000,10282.000000,10282.000000,1.028200e+04,1.028200e+04
mean,2017.992803,73.931800,65.290147,56.635367,52.916315,0.006203,0.000668,7.92190,0.991225,29.153745,38.440965,-4.793040e-04,-2.936798e-03
std,3.161254,15.114613,14.456391,14.211767,13.773589,0.035767,0.015961,2.25847,1.880956,1.939194,22.484143,7.072098e-01,7.070663e-01
min,2013.000000,11.480000,3.270000,-5.170000,-6.750000,0.000000,0.000000,0.78000,0.000000,0.000000,0.000000,-1.000000e+00,-1.000000e+00
25%,2015.000000,66.382500,58.000000,48.770000,45.230000,0.000000,0.000000,6.57250,0.110000,29.450000,19.000000,-8.660254e-01,-8.660254e-01
50%,2018.000000,76.030000,65.290147,56.635367,52.916315,0.000000,0.000000,7.92190,0.320000,29.700000,38.000000,1.224647e-16,-1.836970e-16
75%,2021.000000,84.130000,75.817500,66.365000,61.247500,0.000000,0.000000,8.87000,0.920000,29.830000,58.000000,5.000000e-01,5.000000e-01
max,2023.000000,112.750000,100.840000,94.470000,80.780000,0.700000,0.720000,19.28000,17.120000,30.430000,77.000000,1.000000e+00,1.000000e+00


In [174]:
data.head()

,Year,Max Temperature,Avg Temperature,Min Temperature,Dew Point,Precipitation,Snowdepth,Wind,Gust Wind,Sea Level Pressure,City_Encoded,Month_sin,Month_cos
Date,,,,,,,,,,,,,
2013-01-01,2013,76.81,66.86,56.81,52.39,0.0,0.0,8.21,0.06,29.95,0,0.500000,8.660254e-01
2013-02-01,2013,78.93,68.66,59.57,55.58,0.0,0.0,8.25,0.08,29.90,0,0.866025,5.000000e-01
2013-03-01,2013,86.13,74.63,64.68,53.01,0.0,0.0,8.90,0.12,29.82,0,1.000000,6.123234e-17
2013-04-01,2013,92.63,82.20,68.43,55.99,0.0,0.0,8.76,0.09,29.68,0,0.866025,-5.000000e-01
2013-05-01,2013,98.52,87.53,77.16,62.66,0.0,0.0,9.17,0.13,29.60,0,0.500000,-8.660254e-01


In [175]:
features = ['Max Temperature', 'Min Temperature', 'Dew Point', 'Precipitation', 'Snowdepth', 'Wind', 'Gust Wind', 'Sea Level Pressure', 'Month_sin', 'Month_cos']
target = 'Avg Temperature'

X = data[features].values
y = data[target].values

In [176]:
feature_scaler = MinMaxScaler(feature_range=(-1, 1))
target_scaler = MinMaxScaler(feature_range=(-1, 1))

X_scaled = feature_scaler.fit_transform(X)
y_scaled = target_scaler.fit_transform(y.reshape(-1, 1))

In [177]:
X_scaled = np.column_stack((X_scaled, data['City_Encoded']))

In [178]:
X_scaled.shape

(10282, 11)

In [ ]:
sequence_length = 24

X_sequences, y_sequences = [], []
for i in range(len(X_scaled) - sequence_length):
    X_sequences.append(X_scaled[i:i + sequence_length])
    y_sequences.append(y_scaled[i + sequence_length])

X_sequences = np.array(X_sequences)
y_sequences = np.array(y_sequences)

In [180]:
X_train, X_test, y_train, y_test = train_test_split(X_sequences, y_sequences, test_size=0.2, random_state=42)

In [181]:
print(X_train)

[[[ 2.47161055e-01  2.69971899e-01  3.80783731e-01 ...  1.22464680e-16
   -1.00000000e+00  4.00000000e+00]
  [ 1.75471512e-01  2.73183460e-01  4.27167828e-01 ... -5.00000000e-01
   -8.66025404e-01  4.00000000e+00]
  [ 1.61449590e-01  2.63348053e-01  4.66925625e-01 ... -8.66025404e-01
   -5.00000000e-01  4.00000000e+00]
  ...
  [-7.43556828e-02  3.81372943e-03  1.16417228e-01 ...  1.00000000e+00
    6.12323400e-17  5.00000000e+00]
  [ 9.39073763e-02  4.85748695e-02  1.20073118e-01 ...  8.66025404e-01
   -5.00000000e-01  5.00000000e+00]
  [ 3.25762812e-01  2.48494580e-01  2.95327316e-01 ...  5.00000000e-01
   -8.66025404e-01  5.00000000e+00]]

 [[ 2.74809914e-01  2.40573407e-01  3.63334059e-01 ... -1.00000000e+00
   -1.83697020e-16  6.00000000e+00]
  [ 2.74809914e-01  2.40573407e-01  3.63334059e-01 ... -8.66025404e-01
    5.00000000e-01  6.00000000e+00]
  [ 2.74809914e-01  2.40573407e-01  3.63334059e-01 ... -5.00000000e-01
    8.66025404e-01  6.00000000e+00]
  ...
  [ 2.74809914e-01  2.4

In [182]:
print(y_train)

[[0.43527724]
 [0.27129542]
 [0.06405657]
 ...
 [0.26555294]
 [0.27129542]
 [0.22804141]]


In [183]:
print("X_train --> ", X_train.shape)
print("y_train shape --> ", y_train.shape)

X_train -->  (8196, 36, 11)
y_train shape -->  (8196, 1)


RNN

In [184]:
model_RNN = Sequential()

model_RNN.add(SimpleRNN(units=50, activation='tanh', return_sequences=True, input_shape=(30, 11)))
model_RNN.add(Dropout(0.2))

model_RNN.add(SimpleRNN(units=50, activation='tanh', return_sequences=True))
model_RNN.add(Dropout(0.2))

model_RNN.add(SimpleRNN(units=50, activation='tanh', return_sequences=True))
model_RNN.add(Dropout(0.2))

model_RNN.add(SimpleRNN(units=50, activation='tanh'))
model_RNN.add(Dropout(0.2))

model_RNN.add(Dense(units=1))

model_RNN.compile(optimizer='adam', loss='mean_squared_error')

c:\Users\abdul\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.



In [185]:
model_RNN.summary()

Model: "sequential_10"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ simple_rnn_20 (SimpleRNN)       │ (None, 30, 50)         │         3,100 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_40 (Dropout)            │ (None, 30, 50)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ simple_rnn_21 (SimpleRNN)       │ (None, 30, 50)         │         5,050 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_41 (Dropout)            │ (None, 30, 50)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ simple_rnn_22 (SimpleRNN)       │ (None, 30, 50)         │         5,050 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_42 (Dropout)            │ (None, 30, 50)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ simple_rnn_23 (SimpleRNN)       │ (None, 50)             │         5,050 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_43 (Dropout)            │ (None, 50)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_10 (Dense)                │ (None, 1)              │            51 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 18,301 (71.49 KB)

 Trainable params: 18,301 (71.49 KB)

 Non-trainable params: 0 (0.00 B)

In [186]:
model_RNN.compile(optimizer= "adam", loss = "mean_squared_error")

In [187]:
epochs = 200 
batch_size = 32

In [188]:
early_stopping = EarlyStopping(
    monitor='val_loss',    
    patience=10,       
    restore_best_weights=True 
)

reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',    
    factor=0.5,            
    patience=5,           
    min_lr=1e-6           
)

history = model_RNN.fit(
    X_train, y_train,
    epochs=epochs,
    batch_size=batch_size,
    validation_split=0.2,    
    callbacks=[early_stopping, reduce_lr] 
)

Epoch 1/200


205/205 ━━━━━━━━━━━━━━━━━━━━ 10s 24ms/step - loss: 0.4588 - val_loss: 0.0947 - learning_rate: 0.0010
Epoch 2/200
205/205 ━━━━━━━━━━━━━━━━━━━━ 4s 21ms/step - loss: 0.1886 - val_loss: 0.0857 - learning_rate: 0.0010
Epoch 3/200
205/205 ━━━━━━━━━━━━━━━━━━━━ 6s 28ms/step - loss: 0.1263 - val_loss: 0.0716 - learning_rate: 0.0010
Epoch 4/200
205/205 ━━━━━━━━━━━━━━━━━━━━ 6s 27ms/step - loss: 0.0875 - val_loss: 0.0343 - learning_rate: 0.0010
Epoch 5/200
205/205 ━━━━━━━━━━━━━━━━━━━━ 4s 20ms/step - loss: 0.0576 - val_loss: 0.0311 - learning_rate: 0.0010
Epoch 6/200
205/205 ━━━━━━━━━━━━━━━━━━━━ 4s 22ms/step - loss: 0.0446 - val_loss: 0.0228 - learning_rate: 0.0010
Epoch 7/200
205/205 ━━━━━━━━━━━━━━━━━━━━ 4s 20ms/step - loss: 0.0344 - val_loss: 0.0221 - learning_rate: 0.0010
Epoch 8/200
205/205 ━━━━━━━━━━━━━━━━━━━━ 4s 21ms/step - loss: 0.0293 - val_loss: 0.0268 - learning_rate: 0.0010
Epoch 9/200
205/205 ━━━━━━━━━━━━━━━━━━━━ 5s 24ms/step - loss: 0.0242 - val_loss: 0.0205 - learning_rate: 0.0010
Epo

In [189]:
train_loss = history.history['loss']
val_loss = history.history['val_loss']
epochs_range = list(range(1, epochs + 1))

In [190]:
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=epochs_range,
    y=train_loss,
    mode='lines',
    name='Training Loss'
))

fig.add_trace(go.Scatter(
    x=epochs_range,
    y=val_loss,
    mode='lines',
    name='Validation Loss'
))

fig.update_layout(
    title='Training and Validation Loss (Gradient Descent)',
    xaxis_title='Epochs',
    yaxis_title='Loss',
    template='plotly_white'
)

fig.show()

In [191]:
y_pred_scaled_RNN = model_RNN.predict(X_test)

y_pred_RNN = target_scaler.inverse_transform(y_pred_scaled_RNN)
y_actual_RNN = target_scaler.inverse_transform(y_test)

y_pred_RNN = y_pred_RNN.flatten()
y_actual_RNN = y_actual_RNN.flatten()

mse_RNN = mean_squared_error(y_actual_RNN, y_pred_RNN)
rmse_RNN = np.sqrt(mse_RNN)
mae_RNN = mean_absolute_error(y_actual_RNN, y_pred_RNN)
mape_RNN = mean_absolute_percentage_error(y_actual_RNN, y_pred_RNN) * 100
r2_RNN = r2_score(y_actual_RNN, y_pred_RNN)

fig1 = go.Figure()
fig1.add_trace(go.Scatter(x=np.arange(len(y_actual_RNN)), y=y_actual_RNN, mode='lines', name='Actual', line=dict(color='blue')))
fig1.add_trace(go.Scatter(x=np.arange(len(y_pred_RNN)), y=y_pred_RNN, mode='lines', name='Predicted', line=dict(color='red')))
fig1.update_layout(
    title="Actual vs. Predicted Values",
    xaxis_title="Sample Index",
    yaxis_title="Value",
    legend=dict(x=0, y=1),
    height=400,
    width=800,
    template="plotly_white"
)

errors = y_actual_RNN - y_pred_RNN
fig2 = px.histogram(errors, nbins=30, title="Error Distribution (Actual - Predicted)")
fig2.update_layout(
    xaxis_title="Error",
    yaxis_title="Count",
    height=400,
    width=800,
    template="plotly_white"
)

fig3 = px.scatter(
    x=y_actual_RNN,
    y=y_pred_RNN,
    labels={'x': 'Actual Values', 'y': 'Predicted Values'},
    title="Scatter Plot: Actual vs. Predicted",
)

fig3.update_traces(marker=dict(color='blue', size=8), selector=dict(mode='markers'))

fig3.add_trace(
    px.scatter(
        x=y_actual_RNN, 
        y=y_actual_RNN, 
        labels={'x': 'Actual Values', 'y': 'Predicted Values'},
        title="Scatter Plot: Actual vs. Predicted"
    ).data[0].update(marker=dict(color='red', size=8))
)

fig3.update_layout(height=400, width=800, template="plotly_white")

metrics_text = f"""
**Metrics Summary**  
- Mean Squared Error (MSE_RNN): {mse_RNN:.4f}  
- Root Mean Squared Error (RMSE_RNN): {rmse_RNN:.4f}  
- Mean Absolute Error (MAE_RNN): {mae_RNN:.4f}  
- Mean Absolute Percentage Error (MAPE_RNN): {mape_RNN:.2f}%  
- R² Score: {r2_RNN:.4f}  
"""

metrics_table = go.Figure(data=[go.Table(
    header=dict(values=["Metric", "Value"], fill_color="darkgrey", align="left"),
    cells=dict(values=[
        ["MSE_RNN", "RMSE_RNN", "MAE_RNN", "MAPE_RNN", "R²"],
        [f"{mse_RNN:.4f}", f"{rmse_RNN:.4f}", f"{mae_RNN:.4f}", f"{mape_RNN:.2f}%", f"{r2_RNN:.4f}"]
    ], fill_color="black", align="left"))
])
metrics_table.update_layout(title="Model Performance Metrics", height=300, width=600, template="plotly_dark")

fig1.show()
fig2.show()
fig3.show()
metrics_table.show()


65/65 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step


In [192]:
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=np.arange(100), 
    y=y_actual_RNN[:100],
    mode='lines+markers',
    name='Actual Values',
    marker=dict(symbol='circle', size=6),
    line=dict(width=2)
))

fig.add_trace(go.Scatter(
    x=np.arange(100), 
    y=y_pred_RNN[:100],
    mode='lines+markers',
    name='Predicted Values',
    marker=dict(symbol='x', size=6),
    line=dict(width=2)
))

fig.update_layout(
    title="Actual vs Predicted Values (Sample)",
    xaxis_title="Sample Index",
    yaxis_title="Value",
    template="plotly_white",
    showlegend=True
)

fig.show()

LSTM

In [193]:
model_LSTM = Sequential()

model_LSTM.add(LSTM(units=50, activation='tanh', return_sequences=True, input_shape=(30, 11)))
model_LSTM.add(Dropout(0.2))

model_LSTM.add(LSTM(units=50, activation='tanh', return_sequences=True))
model_LSTM.add(Dropout(0.2))

model_LSTM.add(LSTM(units=50, activation='tanh', return_sequences=True))
model_LSTM.add(Dropout(0.2))

model_LSTM.add(LSTM(units=50, activation='tanh'))
model_LSTM.add(Dropout(0.2))

model_LSTM.add(Dense(units=1))

model_LSTM.compile(optimizer='adam', loss='mean_squared_error')

c:\Users\abdul\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.



In [194]:
model_LSTM.summary()

Model: "sequential_11"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_20 (LSTM)                  │ (None, 30, 50)         │        12,400 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_44 (Dropout)            │ (None, 30, 50)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_21 (LSTM)                  │ (None, 30, 50)         │        20,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_45 (Dropout)            │ (None, 30, 50)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_22 (LSTM)                  │ (None, 30, 50)         │        20,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_46 (Dropout)            │ (None, 30, 50)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_23 (LSTM)                  │ (None, 50)             │        20,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_47 (Dropout)            │ (None, 50)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_11 (Dense)                │ (None, 1)              │            51 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 73,051 (285.36 KB)

 Trainable params: 73,051 (285.36 KB)

 Non-trainable params: 0 (0.00 B)

In [195]:
model_LSTM.compile(optimizer= "adam", loss = "mean_squared_error")

In [196]:
epochs = 200 
batch_size = 32

In [197]:
early_stopping = EarlyStopping(
    monitor='val_loss',    
    patience=10,       
    restore_best_weights=True 
)

reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',    
    factor=0.5,            
    patience=5,           
    min_lr=1e-6           
)

history = model_LSTM.fit(
    X_train, y_train,
    epochs=epochs,
    batch_size=batch_size,
    validation_split=0.2,    
    callbacks=[early_stopping, reduce_lr] 
)

Epoch 1/200
205/205 ━━━━━━━━━━━━━━━━━━━━ 15s 44ms/step - loss: 0.0904 - val_loss: 0.0574 - learning_rate: 0.0010
Epoch 2/200
205/205 ━━━━━━━━━━━━━━━━━━━━ 8s 41ms/step - loss: 0.0541 - val_loss: 0.0354 - learning_rate: 0.0010
Epoch 3/200
205/205 ━━━━━━━━━━━━━━━━━━━━ 10s 47ms/step - loss: 0.0358 - val_loss: 0.0276 - learning_rate: 0.0010
Epoch 4/200
205/205 ━━━━━━━━━━━━━━━━━━━━ 9s 42ms/step - loss: 0.0301 - val_loss: 0.0237 - learning_rate: 0.0010
Epoch 5/200
205/205 ━━━━━━━━━━━━━━━━━━━━ 8s 41ms/step - loss: 0.0261 - val_loss: 0.0211 - learning_rate: 0.0010
Epoch 6/200
205/205 ━━━━━━━━━━━━━━━━━━━━ 9s 46ms/step - loss: 0.0222 - val_loss: 0.0175 - learning_rate: 0.0010
Epoch 7/200
205/205 ━━━━━━━━━━━━━━━━━━━━ 9s 42ms/step - loss: 0.0201 - val_loss: 0.0157 - learning_rate: 0.0010
Epoch 8/200
205/205 ━━━━━━━━━━━━━━━━━━━━ 9s 42ms/step - loss: 0.0180 - val_loss: 0.0133 - learning_rate: 0.0010
Epoch 9/200
205/205 ━━━━━━━━━━━━━━━━━━━━ 9s 44ms/step - loss: 0.0148 - val_loss: 0.0133 - learning_rat

In [198]:
train_loss = history.history['loss']
val_loss = history.history['val_loss']
epochs_range = list(range(1, epochs + 1))

In [199]:
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=epochs_range,
    y=train_loss,
    mode='lines',
    name='Training Loss'
))

fig.add_trace(go.Scatter(
    x=epochs_range,
    y=val_loss,
    mode='lines',
    name='Validation Loss'
))

fig.update_layout(
    title='Training and Validation Loss (Gradient Descent)',
    xaxis_title='Epochs',
    yaxis_title='Loss',
    template='plotly_white'
)

fig.show()

In [200]:
y_pred_scaled_LSTM = model_LSTM.predict(X_test)

y_pred_LSTM = target_scaler.inverse_transform(y_pred_scaled_LSTM)
y_actual_LSTM = target_scaler.inverse_transform(y_test)

y_pred_LSTM = y_pred_LSTM.flatten()
y_actual_LSTM = y_actual_LSTM.flatten()

mse_LSTM = mean_squared_error(y_actual_LSTM, y_pred_LSTM)
rmse_LSTM = np.sqrt(mse_LSTM)
mae_LSTM = mean_absolute_error(y_actual_LSTM, y_pred_LSTM)
mape_LSTM = mean_absolute_percentage_error(y_actual_LSTM, y_pred_LSTM) * 100
r2_LSTM = r2_score(y_actual_LSTM, y_pred_LSTM)

fig1 = go.Figure()
fig1.add_trace(go.Scatter(x=np.arange(len(y_actual_LSTM)), y=y_actual_LSTM, mode='lines', name='Actual', line=dict(color='blue')))
fig1.add_trace(go.Scatter(x=np.arange(len(y_pred_LSTM)), y=y_pred_LSTM, mode='lines', name='Predicted', line=dict(color='red')))
fig1.update_layout(
    title="Actual vs. Predicted Values",
    xaxis_title="Sample Index",
    yaxis_title="Value",
    legend=dict(x=0, y=1),
    height=400,
    width=800,
    template="plotly_white",
)

errors = y_actual_LSTM - y_pred_LSTM
fig2 = px.histogram(errors, nbins=30, title="Error Distribution (Actual - Predicted)")
fig2.update_layout(
    xaxis_title="Error",
    yaxis_title="Count",
    height=400,
    width=800,
    template="plotly_white"
)


fig3 = px.scatter(
    x=y_actual_RNN,
    y=y_pred_RNN,
    labels={'x': 'Actual Values', 'y': 'Predicted Values'},
    title="Scatter Plot: Actual vs. Predicted",
)

fig3.update_traces(marker=dict(color='blue', size=8), selector=dict(mode='markers'))

fig3.add_trace(
    px.scatter(
        x=y_actual_RNN, 
        y=y_actual_RNN, 
        labels={'x': 'Actual Values', 'y': 'Predicted Values'},
        title="Scatter Plot: Actual vs. Predicted"
    ).data[0].update(marker=dict(color='red', size=8))
)

fig3.update_layout(height=400, width=800, template="plotly_white")



metrics_text = f"""
**Metrics Summary**  
- Mean Squared Error (MSE_LSTM): {mse_LSTM:.4f}  
- Root Mean Squared Error (RMSE_LSTM): {rmse_LSTM:.4f}  
- Mean Absolute Error (MAE_LSTM): {mae_LSTM:.4f}  
- Mean Absolute Percentage Error (MAPE_LSTM): {mape_LSTM:.2f}%  
- R² Score: {r2_LSTM:.4f}  
"""

metrics_table = go.Figure(data=[go.Table(
    header=dict(values=["Metric", "Value"], fill_color="darkgrey", align="left"),
    cells=dict(values=[
        ["MSE_LSTM", "RMSE_LSTM", "MAE_LSTM", "MAPE_LSTM", "R²"],
        [f"{mse_LSTM:.4f}", f"{rmse_LSTM:.4f}", f"{mae_LSTM:.4f}", f"{mape_LSTM:.2f}%", f"{r2_LSTM:.4f}"]
    ], fill_color="black", align="left"))
])
metrics_table.update_layout(title="Model Performance Metrics", height=300, width=600, template="plotly_dark",)

fig1.show()
fig2.show()
fig3.show()
metrics_table.show()


65/65 ━━━━━━━━━━━━━━━━━━━━ 2s 26ms/step


In [201]:
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=np.arange(100), 
    y=y_actual_LSTM[:100],
    mode='lines+markers',
    name='Actual Values',
    marker=dict(symbol='circle', size=6),
    line=dict(width=2)
))

fig.add_trace(go.Scatter(
    x=np.arange(100), 
    y=y_pred_LSTM[:100],
    mode='lines+markers',
    name='Predicted Values',
    marker=dict(symbol='x', size=6),
    line=dict(width=2)
))

fig.update_layout(
    title="Actual vs Predicted Values (Sample)",
    xaxis_title="Sample Index",
    yaxis_title="Value",
    template="plotly_white",
    showlegend=True
)

fig.show()

In [202]:
metrics = ["MSE", "RMSE", "MAE", "MAPE (%)", "R²"]
LSTM_metrics = [mse_LSTM, rmse_LSTM, mae_LSTM, mape_LSTM, r2_LSTM]
RNN_metrics = [mse_RNN, rmse_RNN, mae_RNN, mape_RNN, r2_RNN]

comparison_df = pd.DataFrame({
    "Metric": metrics,
    "LSTM": LSTM_metrics,
    "RNN": RNN_metrics
})

fig_metrics = go.Figure()
fig_metrics.add_trace(go.Bar(
    x=comparison_df["Metric"], 
    y=comparison_df["LSTM"], 
    name="LSTM", 
    marker_color='red'
))
fig_metrics.add_trace(go.Bar(
    x=comparison_df["Metric"], 
    y=comparison_df["RNN"], 
    name="RNN", 
    marker_color='blue'
))
fig_metrics.update_layout(
    title="Comparison of Metrics: LSTM vs RNN",
    xaxis_title="Metric",
    yaxis_title="Value",
    barmode="group",
    legend=dict(
        x=1,  
        y=1,    
        yanchor='top'  
    ),
    height=400,
    width=800,
    template="plotly_white"
)

fig_predictions = go.Figure()

fig_predictions.add_trace(go.Scatter(
    x=np.arange(len(y_actual_LSTM)), 
    y=y_actual_LSTM, 
    mode='lines', 
    name='LSTM Actual', 
    line=dict(color='blue', dash='solid')
))
fig_predictions.add_trace(go.Scatter(
    x=np.arange(len(y_pred_LSTM)), 
    y=y_pred_LSTM, 
    mode='lines', 
    name='LSTM Predicted', 
    line=dict(color='blue', dash='dot')
))
fig_predictions.add_trace(go.Scatter(
    x=np.arange(len(y_actual_RNN)), 
    y=y_actual_RNN, 
    mode='lines', 
    name='RNN Actual', 
    line=dict(color='red', dash='solid')
))
fig_predictions.add_trace(go.Scatter(
    x=np.arange(len(y_pred_RNN)), 
    y=y_pred_RNN, 
    mode='lines', 
    name='RNN Predicted', 
    line=dict(color='red', dash='dot')
))
fig_predictions.update_layout(
    title="Comparison of Actual vs Predicted: LSTM vs RNN",
    xaxis_title="Sample Index",
    yaxis_title="Value",
    legend=dict(
        x=1, 
        y=1,     
        yanchor='top'  
    ),
    height=400,
    width=800,
    template="plotly_white"
)

fig_metrics.show()
fig_predictions.show()
